# ETL

In [1]:
import pandas as pd
import json
import re
import ast
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\stdio\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

#### Funciones

In [3]:
def reemplazar_nan_con_none(cadena):
    resultado = ""
    indice = 0
    longitud = len(cadena)

    while indice < longitud:
        if cadena[indice:indice+3] == "NaN":
            resultado += "\"NaN\""
            indice += 3
        else:
            resultado += cadena[indice]
            indice += 1

    return resultado

In [4]:
def corregir_comillas(cadena):
    # Buscar comillas simples incorrectamente formateadas dentro de la cadena
    partes = cadena.split('"')
    for i in range(1, len(partes), 2):
        partes[i] = partes[i].replace("'", '"')
    # Volver a unir las partes corregidas
    cadena_corregida = '"'.join(partes)
    return cadena_corregida

In [5]:
def extraer_valores(cadena):
    valores = {}
    
    # Expresiones regulares para cada clave
    patrones = {
        'user_id': r'"user_id"\s*:\s*"([^"]+)"',
        'items_count': r'"items_count"\s*:\s*([^,]+)',
        'steam_id': r'"steam_id"\s*:\s*"([^"]+)"',
        'user_url': r'"user_url"\s*:\s*"([^"]+)"',
        'items': r'"items"\s*:\s*\{([^}]+)\}'
    }
    
    for clave, patron in patrones.items():
        coincidencias = re.search(patron, cadena)
        if coincidencias:
            valores[clave] = coincidencias.group(1)
    
    return valores

In [6]:
def procesar_diccionarios(items_parte):
    # Utiliza una expresión regular para encontrar todas las coincidencias de los diccionarios dentro de corchetes.
    diccionarios = re.findall(r'\{[^}]+\}', items_parte)

    # Inicializa una lista para almacenar los diccionarios procesados.
    resultado = []

    # Define una función para procesar cada diccionario.
    def procesar_diccionario(diccionario_str):
        # Utiliza una expresión regular para extraer los valores de las claves deseadas.
        item_id_match = re.search(r'"item_id": "([^"]+)"', diccionario_str)
        item_name_match = re.search(r'"item_name": "([^"]+)"', diccionario_str)
        playtime_forever_match = re.search(r'"playtime_forever": (\d+)', diccionario_str)
        playtime_2weeks_match = re.search(r'"playtime_2weeks": (\d+)', diccionario_str)

        # Verifica si se encontró una coincidencia para cada clave antes de extraer el valor.
        item_id = item_id_match.group(1) if item_id_match else None
        item_name = item_name_match.group(1) if item_name_match else None
        playtime_forever = int(playtime_forever_match.group(1)) if playtime_forever_match else None
        playtime_2weeks = int(playtime_2weeks_match.group(1)) if playtime_2weeks_match else None

        # Crea un diccionario con los valores extraídos.
        diccionario_resultado = {
            "item_id": item_id,
            "item_name": item_name,
            "playtime_forever": playtime_forever,
            "playtime_2weeks": playtime_2weeks
        }

        return diccionario_resultado

    # Procesa cada diccionario encontrado y agrégalo a la lista de resultados.
    for diccionario_str in diccionarios:
        resultado.append(procesar_diccionario(diccionario_str))

    # El resultado es una lista de diccionarios.
    return resultado


In [7]:
# Función para convertir fechas al formato "YYYY-MM-DD"
def convertir_fecha(fecha):
    # Utilizar expresión regular para extraer componentes de la fecha
    match = re.match(r"Posted (\w+) (\d+), (\d+)", fecha)
    if match:
        mes_str, dia_str, anio_str = match.groups()
        # Mapear nombres de meses a números
        meses = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04',
            'May': '05', 'June': '06', 'July': '07', 'August': '08',
            'September': '09', 'October': '10', 'November': '11', 'December': '12'
        }
        # Crear la fecha en el nuevo formato
        nueva_fecha = f"{anio_str}-{meses[mes_str]}-{dia_str.zfill(2)}"
        return nueva_fecha
    else:
        return None

#### Cargar australian_user_reviews.json y convertirlo a parquet

In [8]:
with open('../dataset/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data_list = []
    for linea in file:
        linea = reemplazar_nan_con_none(linea)
        data = ast.literal_eval(linea.strip())
        if isinstance(data, dict):
            data_list.append(data)
    reviews = pd.DataFrame(data_list)

In [9]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [10]:
reviews['user_id'].value_counts() #Validar duplicados

user_id
76561198027488037    3
76561198045953692    3
76561198051777058    3
76561198100326818    3
blablabla174         3
                    ..
SakurasouNo          1
goneckahorse         1
coutlindo            1
superdedicated       1
LydiaMorley          1
Name: count, Length: 25485, dtype: int64

In [11]:
reviews = reviews.drop_duplicates(subset=['user_id']) #Eliminar duplicados

Función analisis de sentimiento

In [12]:
sia = SentimentIntensityAnalyzer()
def analyze_sentiment(lista):
    nlist =[]
    for i in lista:
        sentiment = sia.polarity_scores(i['review'])    
        try:               
            if sentiment['compound'] >= 0.05:
                i['sentiment_analysis'] = 2  # Positivo
                del i['review']
            elif sentiment['compound'] <= -0.05:
                i['sentiment_analysis'] = 0  # Malo
                del i['review']
            else:
                i['sentiment_analysis'] = 1  # Neutral
                del i['review']
        except:
            i['sentiment_analysis'] = 1
            del i['review']
        nlist.append(i)
    return nlist

In [13]:
reviews['reviews'] = reviews['reviews'].apply(lambda x: analyze_sentiment(x))   # Aplicando NLP

In [14]:
def desanidar_reviews(df):
    i = 0
    data_list = []
    while i <= len(df['user_id']) -1:
        user_id = df['user_id'].iloc[i]
        user_url = df['user_url'].iloc[i]
        lista = df['reviews'].iloc[i]
        for j in lista:
            j['user_id'] = user_id
            j['user_url'] = user_url
            data_list.append(j)
        i = i + 1
    return data_list

In [15]:
n_reviews = pd.DataFrame(desanidar_reviews(reviews))

In [16]:
# Aplicar la función de conversión a la columna y crear una nueva columna con las fechas reformateadas
n_reviews['posted_date'] = n_reviews['posted'].apply(convertir_fecha)
n_reviews['posted_date'] = pd.to_datetime(n_reviews['posted_date'])

In [17]:
n_reviews['posted_date'].describe()

count                            48498
mean     2014-09-07 21:14:04.884325120
min                2010-10-16 00:00:00
25%                2014-02-19 00:00:00
50%                2014-09-09 00:00:00
75%                2015-05-02 00:00:00
max                2015-12-31 00:00:00
Name: posted_date, dtype: object

Guarda DF en archivo parquet

In [18]:
n_reviews.to_parquet('../dataset/australian_user_reviews.parquet', engine='pyarrow', compression='snappy')

#### Cargar output_steam_games.json y convertirlo a parquet

In [19]:
with open('../dataset/output_steam_games.json', 'r', encoding='utf-8') as file:
    data_list = []
    for linea in file:
        linea = linea.replace('"NaN"', '')
        data = json.loads(linea.strip())
        if isinstance(data, dict):
            data_list.append(data)
    games = pd.DataFrame(data_list)
    games['price'] = games['price'].replace('Free To Play', 0)
    games['price'] = pd.to_numeric(games['price'], errors='coerce')
    games['metascore'] = pd.to_numeric(games['metascore'], errors='coerce')

In [20]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           29310 non-null  float64
 11  early_access    32135 non-null  object 
 12  id              32133 non-null  object 
 13  metascore       2607 non-null   float64
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  object 
 17  items           88310 non-nul

In [21]:
games['id'].value_counts()         # Validando registros duplicados

id
612880    2
761140    1
530200    1
518690    1
513460    1
         ..
676060    1
494160    1
215280    1
667090    1
681550    1
Name: count, Length: 32132, dtype: int64

In [22]:
games = games.drop_duplicates(subset=['id'])

In [23]:
# Inputar cero a valores nulos de las columnas 'price' y 'discount_price'
games[['price','discount_price']] = games[['price','discount_price']].fillna(0) 

In [24]:
# Eliminar nulos para aquellos registros que tengan regitros nulo en las columnas 'title', 'app_name' e 'id
games = games.dropna(subset=['title', 'app_name','id'], how='all')

Guardar output_steam_games.parquet

In [25]:
games.to_parquet('../dataset/output_steam_games.parquet', engine='pyarrow', compression='snappy')

### Cargar australian_users_items.json y convertirlo a parquet

In [26]:
items = pd.DataFrame(columns=['user_id', 'items_count','steam_id','user_url','items'])
with open('../dataset/australian_users_items.json', 'r', encoding='utf-8') as file:
    data_list = []
    for linea in file:
        linea = linea.replace('"NaN"', '')
        linea = linea.replace('\'', '\"')
        resultado = re.search(r'"items":\s*(.+)', linea)
        linea = extraer_valores(linea)
        items_parte = resultado.group(1)
        items_parte = procesar_diccionarios(items_parte)
        linea['items'] = items_parte 
        data_list.append(linea)

items = pd.DataFrame(data_list)

In [27]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  object
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: object(5)
memory usage: 3.4+ MB


In [28]:
items['user_id'].value_counts()         #Validando registros duplicados

user_id
X03-Suits            3
76561198027488037    3
76561198100326818    3
76561198309337430    3
76561198051777058    3
                    ..
8392158              1
76561198056804863    1
SparklezTheTurtle    1
76561198019707497    1
edward_tremethick    1
Name: count, Length: 87626, dtype: int64

In [29]:
items = items.drop_duplicates(subset=['user_id'])       # Eliminar duplicados

In [30]:
items.iloc[0]

user_id                                        76561197970982479
items_count                                                  277
steam_id                                       76561197970982479
user_url       http://steamcommunity.com/profiles/76561197970...
items          [{'item_id': '10', 'item_name': 'Counter-Strik...
Name: 0, dtype: object

In [31]:
def desanidar_items(df):
    i = 0
    data_list = []
    while i <= len(df['user_id']) - 1:
        user_id = df['user_id'].iloc[i]
        steam_id = df['steam_id'].iloc[i]
        user_url = df['user_url'].iloc[i]
        lista = df['items'].iloc[i]
        for j in lista:
            j['user_id'] = user_id
            j['steam_id'] = steam_id
            j['user_url'] = user_url
            data_list.append(j)
        i = i + 1
    return data_list

In [32]:
n_items = pd.DataFrame(desanidar_items(items))

In [33]:
n_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   steam_id          object
 6   user_url          object
dtypes: int64(2), object(5)
memory usage: 272.1+ MB


Guardar australian_users_items.parquet

In [34]:
n_items.to_parquet('../dataset/australian_users_items.parquet', engine='pyarrow', compression='snappy')

# API

In [1]:
import pandas as pd
import json

Cargar dataframes desde el directorio datasets

In [2]:
reviews  = pd.read_parquet('../dataset/australian_user_reviews.parquet')
items =  pd.read_parquet('../dataset/australian_users_items.parquet')
games =  pd.read_parquet('../dataset/output_steam_games.parquet')
ranking_genre = pd.read_parquet('../dataset/ranking_genre.parquet')
usersXgenre = pd.read_parquet('../dataset/user_genre.parquet')

- def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [37]:
def userdata(User_id:str):
    Consumo = 0
    user_data = reviews[reviews['user_id'] == User_id]
    item_count = user_data['user_id'].count()
    for i in user_data['item_id']:
        filtro = games[games['id'] == i]
        precio = filtro.iloc[0]['price'] - filtro.iloc[0]['discount_price']
        Consumo = Consumo + precio
    recomemdaciones = reviews[reviews['user_id'] == User_id]
    cierto = recomemdaciones[recomemdaciones['recommend'] == True].count()
    porcentaje = cierto[0]  / item_count
    return {"Consumo del usuario": Consumo, "% de recomendación": porcentaje * 100, "Cantidad de items": item_count}

In [38]:
userdata('kimjongadam')

{'Consumo del usuario': 54.96,
 '% de recomendación': 80.0,
 'Cantidad de items': 5}

- def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [39]:
def countreviews(fecha1:str,fecha2:str):
    if fecha1 > fecha2:
        return {"Error":"Ingrese correctamente las fechas"}
    else:
        filtro = reviews[(reviews['posted_date'] >= fecha1) & (reviews['posted_date'] <= fecha2) ]
        cierto = filtro[filtro['recommend'] == True].count()
        Tot_rev = filtro['recommend'].count()
        porcentaje = cierto['recommend'] / Tot_rev
    resultado = {"Cantidad de Reviews": int(filtro['recommend'].count()), "% de recomendaciones": round(float(porcentaje),2) * 100}
    return json.dumps(resultado)

In [40]:
countreviews('2010-10-15','2010-12-20')

'{"Cantidad de Reviews": 53, "% de recomendaciones": 98.0}'

- def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

Generar ranking_genre.parquet

In [3]:
# No ejecutar esta celda en render ya que se consume alrededor de 2 Gb de memoria y la versión gratis tiene solo 512 Mb
# Separar lista de generos
games_exploded = games.explode('genres')
# Combinar DF's games_explode con items coincidiendo por id    
merged_data = games_exploded.merge(items, left_on='id', right_on='item_id', how='inner')
# Sumar playtime_forever por género, este dataframe supera el gigabyte de memoria, por ello es necesario almacenar el df ranking_genre
genre_playtime = merged_data.groupby('genres')['playtime_forever'].sum().reset_index()
# Ordenar de mayor a menor sobre 'playtime_forever'
ranking_genre = genre_playtime.sort_values(by='playtime_forever', ascending=False)
ranking_genre = ranking_genre.reset_index(drop=True)
# Guardar df en formato parquet
ranking_genre.to_parquet('../dataset/ranking_genre.parquet', engine='pyarrow', compression='snappy')

In [4]:

def genre(genero:str):
    return {"Puesto numero:": ranking_genre[ranking_genre['genres'] == genero].index[0] + 1 }

In [5]:
genre('Photo Editing')

{'Puesto numero:': 21}

- def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [6]:
# Crear df user_genre.parquet

# Creando df con las columnas 'genres', 'playtime_forever' y 'user_id_y'
usersXgenre = merged_data[['genres','playtime_forever','user_id_y','user_url']]
# Agrupando por usuario y genero
usersXgenre = usersXgenre.groupby(['genres','user_id_y','user_url'])['playtime_forever'].sum().reset_index()
# Guardar df en formato parquet
usersXgenre.to_parquet('../dataset/user_genre.parquet', engine='pyarrow', compression='snappy')

In [8]:
def userforgenre(genero:str):
    filtro = usersXgenre[usersXgenre['genres'] == genero].sort_values(by='playtime_forever', ascending=False).head(5)
    lista = []
    j = 0
    for i in filtro['user_id_y']:
        filtro2 = filtro[filtro['user_id_y'] == i]
        res = {"Posicion": j + 1, "Usuario_Id":i,"Url_Usuario":filtro2['user_url'].iloc[0]}
        lista.append(res)
        j = j + 1
    return lista

In [9]:
userforgenre('RPG')

[{'Posicion': 1,
  'Usuario_Id': 'shinomegami',
  'Url_Usuario': 'http://steamcommunity.com/id/shinomegami'},
 {'Posicion': 2,
  'Usuario_Id': 'tobscene',
  'Url_Usuario': 'http://steamcommunity.com/id/tobscene'},
 {'Posicion': 3,
  'Usuario_Id': 'REBAS_AS_F-T',
  'Url_Usuario': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 {'Posicion': 4,
  'Usuario_Id': 'Evilutional',
  'Url_Usuario': 'http://steamcommunity.com/id/Evilutional'},
 {'Posicion': 5,
  'Usuario_Id': 'triggernyar',
  'Url_Usuario': 'http://steamcommunity.com/id/triggernyar'}]

In [10]:
usersXgenre[usersXgenre['genres'] == 'RPG'].sort_values(by='playtime_forever', ascending=False).head(5)

,genres,user_id_y,user_url,playtime_forever
474083,RPG,shinomegami,http://steamcommunity.com/id/shinomegami,1060592
475670,RPG,tobscene,http://steamcommunity.com/id/tobscene,906936
460076,RPG,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,886204
455455,RPG,Evilutional,http://steamcommunity.com/id/Evilutional,680646
475815,RPG,triggernyar,http://steamcommunity.com/id/triggernyar,608768



- def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:

|Activision | |
|:-------:|:----------:|	
| Año | Contenido Free |
| 2023 |	27% |
| 2022 |	25% |
| xxxx |	xx% |

- def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

--Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}